# Epsim execution / plotting

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle
import itertools
from pathlib import Path
from gengraph import EpsimGraph
from epsim import Epsim

## Simple run
for testing purposes

In [ ]:
# Simple run

starttime = time.time()
epsim_graph = EpsimGraph(k=100000, sigma_office=0.5, perc_split_classes=0.5)
sim = Epsim()
sim.init_from_dicts(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard,
                    epsim_graph.school_nbrs_split, epsim_graph.office_nbrs)
sim.run_sim(sim_iters=400, num_start_nodes=100, num_immunized_nodes=0, start_weekday=1, p_spread_family=0.37, p_spread_school=0.24, 
            p_spread_office=0.24, p_detect_child=0.2, p_detect_parent=0.5, p_testing=0.5, print_progress=True, export_csv=False)
print(f"runtime: {time.time() - starttime}s")

## Multiple runs per parameter combination
for plotting an averaged diagram of multiple runs to avoid outliners

In [ ]:
# Define simulation parameters
sim_iters = 400

def gen_graph_run_sim(k, sigma_office, num_start_nodes, num_immunized_nodes, start_weekday, perc_split_classes, p_spread_family, 
                      p_spread_school, p_spread_office, p_detect_child, p_detect_parent, p_testing):
    epsim_graph = EpsimGraph(k, sigma_office, perc_split_classes)
    sim = Epsim()
    sim.init_from_dicts(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard, epsim_graph.school_nbrs_split, epsim_graph.office_nbrs)
    return sim.run_sim(sim_iters, num_start_nodes, num_immunized_nodes, start_weekday, p_spread_family, p_spread_school, 
                       p_spread_office, p_detect_child, p_detect_parent, p_testing)


sim_params = {
    'k': [100000],
    'sigma_office': [0.5],
    'num_start_nodes': [100],
    'num_immunized_nodes': [0],
    'start_weekday': [0],
    'perc_split_classes': [0.5],
    'p_spread_family': [0.37],
    'p_spread_school': [0.2, 0.22, 0.24],
    'p_spread_office': [0.2, 0.22, 0.24],
    'p_detect_child': [0.2],
    'p_detect_parent': [0.5],
    'p_testing': [0.5]
}

param_combis = list(itertools.product(*sim_params.values()))

In [ ]:
# Execute simulations: multiple runs per parameter combination

num_runs = 100
runs_per_param_combi = {pc: [gen_graph_run_sim(*pc) for i in range(num_runs)] for pc in param_combis}
pickle.dump(runs_per_param_combi, open("runs_per_param_combi.p", 'wb'))

In [ ]:
# Plot mean infections per round over all runs of one parameter combination to avoid outliners

plt.rcParams.update({'font.size': 15})

for pc, runs in runs_per_param_combi.items():
    print(dict(zip(sim_params, pc)))
    infecs_per_runs_per_rounds = zip(*runs)
    mean_infecs_per_round = [np.mean(infecs_per_runs) for infecs_per_runs in infecs_per_runs_per_rounds]
    fig, axs = plt.subplots(figsize=(20,13))
    fig.patch.set_facecolor('xkcd:white')
    axs.set_ylim(top=200000)
    axs.set_xlim(0, sim_iters)
    axs.plot(range(sim_iters), mean_infecs_per_round)
    plt.annotate(max(mean_infecs_per_round), xy=(1, max(mean_infecs_per_round)), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    axs.set_xlabel("Rounds")
    axs.set_ylabel(f"Infected nodes (mean of {num_runs} runs)")
    plt.show()
    print()

In [ ]:
# Plot histogram of total infections for all runs of one parameter combination
# This shows how much the total infections of different simulations with the same parameters deviatie due to the randomness of the simulation

plt.rcParams.update({'font.size': 12})

for pc, runs in runs_per_param_combi.items():
    print(dict(zip(sim_params, pc)))
    total_infec_per_run = [max(run) for run in runs]
    fig, axs = plt.subplots(figsize=(10,6))
    fig.patch.set_facecolor('xkcd:white')
    axs.set_xlabel("Total infected nodes")
    axs.set_ylabel("# Runs")
    axs.hist(total_infec_per_run)
    plt.show()
    print()

## Single run per parameter combination
with csv export of detailed statistics per round (export path has to be modified)

In [ ]:
# Run with various parameters

sim_iters = 400
num_start_nodes = 100
num_immunized_nodes = 0
start_weekday = 0
k = 100000
sigma_office_vals = [0.5]
p_spread_family_vals = [0.37]
p_spread_school_vals = [0.2, 0.22, 0.24]
p_spread_office_vals = [0.2, 0.22, 0.24]
p_detect_child_vals = [0.2]
p_detect_parent_vals = [0.5]
p_testing_vals = [0.5]
perc_split_classes = 0.5

results = {}

starttime = time.time()
for sigma_office in sigma_office_vals:
    epsim_graph = EpsimGraph(k, sigma_office, perc_split_classes)
    sim = Epsim()
    sim.init_from_dicts(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard,
                        epsim_graph.school_nbrs_split, epsim_graph.office_nbrs)
    for p_spread_family in p_spread_family_vals:
        for p_detect_child in p_detect_child_vals:
            for p_detect_parent in p_detect_parent_vals:
                for p_testing in p_testing_vals:
                    for p_spread_school in p_spread_school_vals:
                        for p_spread_office in p_spread_office_vals:
                            export_csv = Path(f'csv/p1_{p_spread_family}_p2_{p_spread_school}.csv')
                            sim_res = sim.run_sim(sim_iters, num_start_nodes, num_immunized_nodes, start_weekday, p_spread_family,
                                                  p_spread_school, p_spread_office, p_detect_child, p_detect_parent, p_testing, export_csv=export_csv)
                            results[sigma_office, p_spread_family, p_detect_child, p_detect_parent, p_testing, p_spread_school, \
                                    p_spread_office] = sim_res

print(f"runtime: {time.time() - starttime}s")

In [ ]:
# Plot various parameter runs

plt.rcParams.update({'font.size': 15})

for sigma_office in sigma_office_vals:
    for p_spread_family in p_spread_family_vals:
        for p_detect_child in p_detect_child_vals:
            for p_detect_parent in p_detect_parent_vals:
                for p_testing in p_testing_vals:
                    for p_spread_office in p_spread_office_vals:
                        fig, axs = plt.subplots(figsize=(20,13))
                        fig.patch.set_facecolor('xkcd:white')
                        axs.set_ylim(top=200000)
                        axs.set_xlim(0, sim_iters)
                        for p_spread_school in p_spread_school_vals:
                            result = results[sigma_office, p_spread_family, p_detect_child, p_detect_parent, p_testing, \ 
                                             p_spread_school, p_spread_office]
                            axs.plot(range(len(result)), result, label=p_spread_school)
                            plt.annotate(max(result), xy=(1, max(result)), xytext=(8, 0),
                                        xycoords=('axes fraction', 'data'), textcoords='offset points')
                        axs.set_title(f"p_spread_family={p_spread_family}")
                        axs.set_xlabel("Rounds")
                        axs.set_ylabel("Infected Nodes")
                        text_fixed = f"sigma_office={sigma_office}, p_detect_child={p_detect_child}, " \
                                     + f"p_detect_parent={p_detect_parent}, p_testing={p_testing}, p_spread_office={p_spread_office}"
                        axs.text(0.18, -0.09, text_fixed, transform=axs.transAxes)
                        axs.text(0.18, -0.138, 'p_spread_school =', transform=axs.transAxes)                   
                        axs.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=6)